## RNN Language Model

Sources

[1] 
[2] https://mlexplained.com/2018/02/15/language-modeling-tutorial-in-torchtext-practical-torchtext-part-2/



We have the following datasets available for this task:

- Penn Trebank (originally created for POS tagging)
- WikiText

Before loading our dataset, define how it will be tokenized and preprocessed. To do this, `torchtext` uses `data.Field`. By default, it uses [`spaCy`](https://spacy.io/api/tokenizer) tokenization.

Also, we set an `init_token` and `eos_token` for the begin and end of sentence characters.

In [1]:
import torch
import torchtext
from torchtext import data

TEXT = data.Field(
    tokenizer_language='en',
    lower=True,
    init_token='<sos>',
    eos_token='<eos>',
    batch_first=True,
)

Now, we can load our dataset

In [2]:
from torchtext.datasets import WikiText2
 
train, valid, test = WikiText2.splits(TEXT) 

TEXT.build_vocab(train, vectors="glove.6B.300d")

print(f"We have {len(TEXT.vocab)} tokens in our vocabulary")

We have 28914 tokens in our vocabulary


## Iterator


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32
BPTT_LEN = 30

train_iter, valid_iter, test_iter = data.BPTTIterator.splits(
    (train, valid, test),
    batch_size=BATCH_SIZE,
    bptt_len=BPTT_LEN, # this is where we specify the sequence length
    device=device,
    repeat=False)

In [4]:
import torch.nn as nn

class RNNLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, pad_idx, hidden_size,
                 cell_class=nn.GRU, dropout=0.20):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=PAD_IDX)
        self.rnn = cell_class(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
    def forward(self, inp, hidden=None):
        """
        Inputs are supposed to be just one step (i.e. one letter)
        """
        # inputs = [batch_size, ]
        emb = self.embedding(inp)
        # emb = [batch, embedding_dim]
        # As all my examples are of the same length, there is no use 
        # in packing the input to the RNN
        rnn_outputs, hidden = self.rnn(emb, hidden)
        # hidden = [batch, hidden_dim]
        
        out = self.fc(self.dropout(rnn_outputs))
        # out = [batch, vocab size]

        return out, hidden

Create the Language Model

In [12]:


PAD_IDX = TEXT.vocab.stoi["<pad>"]
UNK_IDX = TEXT.vocab.stoi["<unk>"]
EOS_IDX = TEXT.vocab.stoi["<eos>"]
SOS_IDX = TEXT.vocab.stoi["<sos>"]


## Training 

In [14]:
import torch.optim as optim

HIDDEN_DIM = 256
vocab_size = TEXT.vocab.vectors.shape[0]
embedding_dim = TEXT.vocab.vectors.shape[1]

model = RNNLanguageModel(vocab_size, embedding_dim, hidden_size=HIDDEN_DIM, pad_idx=PAD_IDX, dropout=0.4)

# Set weight for UNK to a random normal
model.embedding.weight.data.copy_(TEXT.vocab.vectors)
model.embedding.weight.data[UNK_IDX] = torch.randn(embedding_dim)


optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

model = model.to(device)
criterion = criterion.to(device)


## An example of calculating the loss
batch = next(iter(train_iter))

preds, _ = model(batch.text)
preds = preds.view(-1, preds.shape[-1])


trg = batch.target.view(-1)
criterion(preds, trg)

tensor(10.2723, device='cuda:0', grad_fn=<NllLossBackward>)

In [15]:
import torch
import numpy as np

def train(model, iterator, optimizer, criterion):
    """
    Trains the model for one full epoch
    """
    epoch_loss = 0
    epoch_perplexity = 0

    model.train()

    for batch in iterator:
        optimizer.zero_grad()
        text = batch.text
        trg = batch.target.view(-1)
        
        preds, _ = model(text)
        preds = preds.view(-1, preds.shape[-1])
        
        loss = criterion(preds, trg)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_perplexity += np.exp(loss.item())
    
    return epoch_loss / len(iterator), epoch_perplexity / len(iterator)


def evaluate(model, iterator, criterion):
    """
    Evaluates the model on the given iterator
    """
    epoch_loss = .0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text = batch.text
            trg = batch.target.view(-1)

            preds, _ = model(text)
            preds = preds.view(-1, preds.shape[-1])
            
            loss = criterion(preds, trg)

            epoch_loss += loss.item()
            
        loss = epoch_loss / len(iterator)
        
        perplexity = np.exp(loss)

    return loss, perplexity

In [16]:
from tqdm.notebook import tqdm
import time

N_EPOCHS = 20

best_valid_loss = float('inf')

early_stopping_tolerance = 3
epochs_without_improvement = 0

model_path = "/tmp/rnn_lang_model.pt"

pbar = tqdm(range(N_EPOCHS), ncols=1000)
for epoch in pbar:
    
    epoch_bar = tqdm(train_iter)
    train_loss, train_perplexity = train(model, epoch_bar, optimizer, criterion)
    valid_loss, valid_perplexity = evaluate(model, valid_iter, criterion)

    
    desc = f' Train Loss: {train_loss:.3f} Perp: {train_perplexity:.2f}'
    desc += f' Val. Loss: {valid_loss:.3f} Perp: {valid_perplexity:.2f}'
    pbar.set_description(desc)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        epochs_without_improvement = 0
        torch.save(model.state_dict(), model_path)
        print(f"Best model so far (Loss {best_valid_loss:.3f} Perp {valid_perplexity:.2f}) saved at {model_path}")
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= early_stopping_tolerance:
            print("Early stopping")
            break


Best model so far (Loss 5.275 Perp 195.45) saved at /tmp/rnn_lang_model.pt



Best model so far (Loss 5.072 Perp 159.52) saved at /tmp/rnn_lang_model.pt



Best model so far (Loss 4.992 Perp 147.29) saved at /tmp/rnn_lang_model.pt



Best model so far (Loss 4.961 Perp 142.69) saved at /tmp/rnn_lang_model.pt



Best model so far (Loss 4.952 Perp 141.40) saved at /tmp/rnn_lang_model.pt



Early stopping


In [18]:
model.load_state_dict(torch.load(model_path))
model.eval()

valid_loss, valid_perplexity = evaluate(model, valid_iter, criterion)
test_loss, test_perplexity = evaluate(model, test_iter, criterion)


print(f"Valid loss      : {valid_loss:.2f}")
print(f"Valid perplexity: {valid_perplexity:.2f}\n")

print(f"Test loss      : {test_loss:.2f}")
print(f"Test perplexity: {test_perplexity:.2f}")

Valid loss      : 4.95
Valid perplexity: 141.40

Test loss      : 4.89
Test perplexity: 132.81


## Sampling

In [19]:
import torch.nn.functional as F

def sample_sentence(init_token="<eos>", temperature=1):

    seq = [TEXT.vocab.stoi[init_token]]

    while len(seq) == 1 or seq[-1] != EOS_IDX:
        inp = torch.LongTensor([[seq[-1]]]).to(device)
        out, _ = model(inp)

        """
        Sample from probabilities
        """
        probs = F.softmax(out.view(-1) / temperature, dim=0)
        next_tok_idx = torch.multinomial(probs, num_samples=1)
        
        seq.append(next_tok_idx)
        
    return [TEXT.vocab.itos[t] for t in seq]

In [20]:
for temperature in np.arange(0.5, 1.5, 0.15):
    print("="*80, f"\nSampling with temperature = {temperature:.2f}")
    
    print(" ".join(sample_sentence("the", temperature=temperature)))

Sampling with temperature = 0.50
the <unk> <unk> <unk> as a large numbers of the golden <unk> , and the british british navy , and the <unk> , and the <unk> , and the <unk> , the <unk> , and <unk> . the <unk> . the player to give only the <unk> <unk> , the dominican republic of the city 's trick 's oxford , and the <unk> , the same day . <eos>
Sampling with temperature = 0.65
the town of the monument , and two <unk> , the song by the king of england in the writers and the <unk> , and a foundation was to take place in the standard and small match . it was an <unk> ( <unk> and the <unk> and was forced the policies of after the advent of the the <unk> . the <unk> the the nation , which and the northwest , but were able to the two <unk> of the first time , <unk> , the game , the country in the central and a new york , <unk> <unk> and <unk> . <eos>
Sampling with temperature = 0.80
the former governor 's baltimore , king gregory of the were upgraded to mark the last large classical music , a

As we rise temperature, we have more variety at the cost of meaningless stuff..

### Hidden State

There is a problem here! We are missing the hidden state

In [21]:
import torch.nn.functional as F

def sample_sentence(init_token="<eos>", temperature=1):

    seq = [TEXT.vocab.stoi[init_token]]
    hidden = None
    while len(seq) == 1 or seq[-1] != EOS_IDX:
        inp = torch.LongTensor([[seq[-1]]]).to(device)
        out, hidden = model(inp, hidden=hidden)

        """
        Sample from probabilities
        """
        probs = F.softmax(out.view(-1) / temperature, dim=0)
        next_tok_idx = torch.multinomial(probs, num_samples=1)
        
        seq.append(next_tok_idx)
        
    return [TEXT.vocab.itos[t] for t in seq]

In [22]:
for temperature in np.arange(0.5, 1.5, 0.15):
    print("="*80, f"\nSampling with temperature = {temperature:.2f}")
    
    print(" ".join(sample_sentence("the", temperature=temperature)))

Sampling with temperature = 0.50
the book was released as a single @-@ @-@ website . <eos>
Sampling with temperature = 0.65
the rest of the city 's war and the development of the " extreme " . <eos>
Sampling with temperature = 0.80
the capitol on the whole of the african american <unk> who began the breeders ' single , directing their box studio director of an acronym . " this is no curious environment , but and all she is the most other part of the great @-@ irish @-@ american author , which was to see the <unk> play at the time of the character . the leading the cast of tintin was to see these direct trials on the le petit vingtième , which had also been commissioned by actress supporters . the film dr. john <unk> . the bell ( <unk> , transverse ) , formed six of the four @-@ twenty @-@ american television stories : " i am unicorn , " and as the end of the same year , so <unk> , the show of the character was also structured in " sentences , which involved the title , " and would be <

We can observe that:

- with hidden states there are more "meaningful" stuff
- quotation marks are closed when using the hidden state